In [104]:
# data engineer
import pandas as pd
import numpy as np
from xlrd import open_workbook
import csv
import json
import math
import queue
FILE_FOLDER = '../prediction_data/%s'
AFTER_FILE_FOLDER = '../prediction_data/afterAnalysis/%s'

# Generate AirQuality stations information from excel  to csv
### new file name as "aq_stations.csv"

In [22]:
wb = open_workbook(FILE_FOLDER%'Beijing_AirQuality_Stations_en.xlsx')
sheet_names = wb.sheet_names()
sheet = wb.sheet_by_name(sheet_names[0])
number_of_rows = sheet.nrows
number_of_columns = sheet.ncols

stationId = []
longitude = []
latitude = []
stationType = [] 
current_type = ''
for row in range(11, number_of_rows):
    if(sheet.cell(row,0).value == ''):
        continue
    elif(row in [12-1,26-1,39-1,48-1]):
        current_type = sheet.cell(row,0).value
    else:
        stationId.append(sheet.cell(row,0).value)
        longitude.append(sheet.cell(row,1).value)
        latitude.append(sheet.cell(row,2).value)
        stationType.append(current_type)
D = {
    'stationId':pd.Series(stationId),
    'latitude':pd.Series(latitude),
    'longitude':pd.Series(longitude),
    'stationType':pd.Series(stationType),
}
df_aq_station = pd.DataFrame(D)
df_aq_station.head()
# df_aq_station.to_csv(AFTER_FILE_FOLDER%'aq_stations.csv',index=False)

,stationId,latitude,longitude,stationType
0,dongsi_aq,39.929,116.417,Urban Stations
1,tiantan_aq,39.886,116.407,Urban Stations
2,guanyuan_aq,39.929,116.339,Urban Stations
3,wanshouxigong_aq,39.878,116.352,Urban Stations
4,aotizhongxin_aq,39.982,116.397,Urban Stations


# Generate weather stations information from json(from html)  to csv
### new file name as "w_stations.csv"

In [43]:
from pprint import pprint
json_data_file = AFTER_FILE_FOLDER%'meo_data.json'
with open(json_data_file) as f:
    data = json.load(f)

w_station_name = []
w_longitude = []
w_latitude = []
for ele in data['data']:
    w_station_name.append(ele['name'].split(':')[0].replace(' ',''))
    w_longitude.append(ele['lnglat'][0])
    w_latitude.append(ele['lnglat'][1])
    
w_d = {
    'stationId':pd.Series(w_station_name),
    'latitude':pd.Series(w_latitude),
    'longitude':pd.Series(w_longitude)
}
df_observed_weather_stations = pd.DataFrame(w_d)
# df_observed_weather_stations.to_csv(AFTER_FILE_FOLDER%'ob_weather_stations.csv',index=False)
df_observed_weather_stations.head()
#     pprint(ele['lnglat'][0])

,stationId,latitude,longitude
0,shunyi_meo,40.126667,116.615278
1,hadian_meo,39.986944,116.290556
2,yanqing_meo,40.449444,115.968889
3,miyun_meo,40.377500,116.864167
4,huairou_meo,40.357778,116.626944


# loading grid information

In [54]:
file_name = FILE_FOLDER%'Beijing_grid_weather_station.csv'
# df_grid_stations = pd.read_csv(FILE_FOLDER%'Beijing_grid_weather_station.csv')
# df_grid_stations.head()

with open(file_name) as csvfile:
    spamreader = csv.reader(csvfile)
    # print(list(spamreader)
    df_grid_stations = pd.DataFrame(data= list(spamreader),columns=['gridStationId','latitude','longitude'])
csvfile.close()
df_grid_stations.head()
# df_grid_stations.to_csv(AFTER_FILE_FOLDER%'grid_stations.csv',index=False)

# 

In [71]:
df_aq_stations = pd.read_csv(AFTER_FILE_FOLDER%'aq_stations.csv')
df_observed_weather_stations = pd.read_csv(AFTER_FILE_FOLDER%'ob_weather_stations.csv')
df_grid_stations = pd.read_csv(AFTER_FILE_FOLDER%'grid_stations.csv')

In [72]:
df_aq_stations.head()

,stationId,latitude,longitude,stationType
0,dongsi_aq,39.929,116.417,Urban Stations
1,tiantan_aq,39.886,116.407,Urban Stations
2,guanyuan_aq,39.929,116.339,Urban Stations
3,wanshouxigong_aq,39.878,116.352,Urban Stations
4,aotizhongxin_aq,39.982,116.397,Urban Stations


In [73]:
df_observed_weather_stations.head()

,stationId,latitude,longitude
0,shunyi_meo,40.126667,116.615278
1,hadian_meo,39.986944,116.290556
2,yanqing_meo,40.449444,115.968889
3,miyun_meo,40.377500,116.864167
4,huairou_meo,40.357778,116.626944


In [85]:
df_grid_stations.head()
df_grid_stations.describe()

,latitude,longitude
count,651.000000,651.000000
mean,40.000000,116.500000
std,0.605996,0.895115
min,39.000000,115.000000
25%,39.500000,115.700000
50%,40.000000,116.500000
75%,40.500000,117.300000
max,41.000000,118.000000


# for each air quality station, select n closest observed weather stations and m m grid weather stations.

#### n + m weather information --> manually analysis ( or other scientific methods) --> air quality station weather information
#### air quality station weather information --> neural network model --> pollution results

In [127]:
def point_dis_square(x1,y1,x2,y2):
    dis = (x1*1.0 - x2) * (x1*1.0 - x2) + (y1*1.0 - y2) * (y1*1.0 - y2) 
    return math.sqrt(dis)

aq_len = len(df_aq_stations.stationId)
ow_len = len(df_observed_weather_stations.stationId)
grid_len = len(df_grid_stations)
# init quantile
# aq_ow_threshold = 0.24
# aq_grid_threshold = 0.5
# aq_ow_dis_square = []
# aq_grid_dis_square = []

closest_ow_num = 3
closest_grid_num = 9

for i in range(aq_len):
    aq_la = df_aq_stations.latitude[i]
    aq_lo = df_aq_stations.longitude[i]
    ow_list = []
    grid_list = []
    ow_q = queue.PriorityQueue()
    grid_q = queue.PriorityQueue()
    for j in range(ow_len):
        ow_la = df_observed_weather_stations.latitude[j]
        ow_lo = df_observed_weather_stations.longitude[j]
        stationId = df_observed_weather_stations.stationId[j]
        y = ((point_dis_square(aq_la,aq_lo,ow_la,ow_lo)))
        if(ow_q.qsize() < closest_ow_num): ow_q.put((-1.0*y,stationId))
        else:
            if(ow_q.queue[0][0] < -1.0*y):
                ow_q.get()
                ow_q.put((-1.0*y,stationId))
#         if(y <= aq_ow_threshold): ow_list.append(df_observed_weather_stations.stationId[j])
#         aq_ow_dis_square.append(y)
        
        
    for j in range(grid_len):
        grid_la = df_grid_stations.latitude[j]
        grid_lo = df_grid_stations.longitude[j]
        stationId = df_grid_stations.gridStationId[j]
        
        y = (point_dis_square(aq_la,aq_lo,grid_la,grid_lo))
        if(grid_q.qsize() < closest_grid_num): grid_q.put((-1.0*y,stationId))
        else:
            if(grid_q.queue[0][0] < -1.0*y):
                grid_q.get()
                grid_q.put((-1.0*y,stationId))
#         if(y <= aq_grid_threshold): grid_list.append(df_grid_stations.gridStationId[j])
#         aq_grid_dis_square.append(y)

#     print(ow_q.get(),ow_q.get(),ow_q.get())
    print(list(map(lambda X:X[1],grid_q.queue)))
#     print(ow_q.qsize(),grid_q.qsize())

['beijing_grid_305', 'beijing_grid_323', 'beijing_grid_283', 'beijing_grid_325', 'beijing_grid_302', 'beijing_grid_304', 'beijing_grid_282', 'beijing_grid_303', 'beijing_grid_324']
['beijing_grid_283', 'beijing_grid_325', 'beijing_grid_304', 'beijing_grid_281', 'beijing_grid_302', 'beijing_grid_324', 'beijing_grid_282', 'beijing_grid_303', 'beijing_grid_323']
['beijing_grid_324', 'beijing_grid_262', 'beijing_grid_261', 'beijing_grid_302', 'beijing_grid_303', 'beijing_grid_282', 'beijing_grid_281', 'beijing_grid_283', 'beijing_grid_304']
['beijing_grid_323', 'beijing_grid_261', 'beijing_grid_283', 'beijing_grid_324', 'beijing_grid_303', 'beijing_grid_282', 'beijing_grid_281', 'beijing_grid_302', 'beijing_grid_304']
['beijing_grid_326', 'beijing_grid_284', 'beijing_grid_305', 'beijing_grid_324', 'beijing_grid_304', 'beijing_grid_283', 'beijing_grid_325', 'beijing_grid_303', 'beijing_grid_282']
['beijing_grid_282', 'beijing_grid_346', 'beijing_grid_323', 'beijing_grid_302', 'beijing_grid_